# ISYE 6501 HW 10

## Jeremy Wong | kwong301@gatech.edu

# Question 14.1

The breast cancer data set breast-cancer-wisconsin.data.txt from http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/  (description at http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29) has missing values.
1.	Use the mean/mode imputation method to impute values for the missing data.
2.	Use regression to impute values for the missing data.
3.	Use regression with perturbation to impute values for the missing data.
4.	(Optional) Compare the results and quality of classification models (e.g., SVM, KNN) build using 
    <br>(1) the data sets from questions 1,2,3; 
    <br>(2) the data that remains after data points with missing values are removed; and 
    <br>(3) the data set when a binary variable is introduced to indicate missing values.


# Answers to 14.1

## Some Notes on Regression with Perturbation

The codes for imputation with mean/mode and regression are included below. I've had a busy week, and didn't have time to complete the optional questions. As for regression with perturbation, I've found the materials in the lecture a little bit obscure and ambiguous so I did some reading (e.g. [this Python implementation](https://www.statsmodels.org/0.8.0/_modules/statsmodels/imputation/mice.html) and Piazza posts) and took some notes here. 

"Perturbation" in this context is a method for setting the parameter estimate in an imputation model. There are two ways to implement jointly with a regression (or classification where applicable) model:

1. Suppose variable $Z$ is missing. We first train a model to fit $Z$ with other variables, say a supervised model $Z = F(X; \theta)$ where the model parameter $\theta$ is known to be from a sampling distribution $g$. We create "perturbations" for $\theta$ by drawing from $\hat{g}$ (sample analogue of $g$). With drawn parameter $\tilde{\theta}$, we impute missing $Z$ with $\tilde{Z} = F(X; \tilde{\theta})$.
2. After training the model $Z = F(X; \theta)$. Estimate or derive the distribution for the residuals $\hat{\varepsilon} = Z - F(X; \hat{\theta})$， $h$. We create "perturbation" for $\varepsilon$ by drawing from $\hat{h}$. With drawn $\tilde{\varepsilon}$ impute missing $Z$ with $F(X; \hat{\theta}) + \tilde{\varepsilon}$.

We will use the second approach to create perturbations. Because of asymptotic normality for the linear regression estimator, we can simply take the error distribution as $\mathcal{N}(0, \hat{\sigma}^2)$ where $\hat{\sigma}^2$ is the empirical variance of the residuals.

Naturally, randomness in $h$ is necessarily captured in $g$ as $\theta$ and $\varepsilon$ are related by a deterministic function $F$ and known covariates $X$. Another class of perturbation method is the "bootstrap" class. In which we set the parameter $\theta$ equal to a fitted parameter vector $\theta_0$ obtained when fitting the conditional model $F$ to a bootstrapped version of the data set.

Heuristically, perturbation can be considered as a method that, despite inducing additional potential errors by tweaking optimally trained models, improves the overall variability of the imputed variable. This can be interpreted as a consequence of the variance of the variable being matched and accounted for, as opposed to just the mean in the pure regression imputation case. [Poulos and Valle (2018)](https://arxiv.org/pdf/1610.09075.pdf) points out that perturbation can be considered as a regularization to improve prediction accuracy.

Even though I don't have the chance to complete part 4, I predict that the regression models will do best. The extent of missing data is probably too small for the difference between perturbed/no perturb to take place. I am also skeptical about using approach (3): a binary variable is introduced to indicate missing values. If the missing mechanism is driven by a bimodal process, say, people tend not to report $Z$ if their $Z$'s are either too large and too small, then it won't be a good idea to treat all missing $Z$ equally. 

# Code for 14.1

In [75]:
library(data.table)
library(caret)
library(ggplot2)
library(mice)
library(magrittr)

df <- fread('./breast-cancer-wisconsin.data.txt')

## Imputation with Mean

In [76]:
df_mean_imp <- copy(df)
df_mean_imp[, V7:=as.numeric(V7)]
df_mean_imp[is.na(V7), V7:=mean(df_mean_imp[, V7], na.rm=TRUE)]

Warning message in eval(jsub, SDenv, parent.frame()):
"NAs introduced by coercion"


## Imputation with Mode

In [77]:
df_mode_imp <- copy(df)
df_mode_imp[, V7:=as.numeric(V7)]
V7_mode <- df_mode_imp[!is.na(V7), V7] %>% table %>% which.max
df_mode_imp[is.na(V7), V7:=V7_mode]

Warning message in eval(jsub, SDenv, parent.frame()):
"NAs introduced by coercion"


## Imputation with Linear Regression

In [80]:
df_reg_imp <- copy(df)
df_reg_imp[, V7:=as.numeric(V7)]

# V1 is ID and V11 is response
reg <- lm(V7 ~ V2 + V3 + V4 + V5 + V6 + V7 + V8 + V9 + V10, data=df_reg_imp, na.action=na.omit)
df_reg_imp[is.na(V7), V7:=predict(reg, df_reg_imp[is.na(V7)])]

Warning message in eval(jsub, SDenv, parent.frame()):
"NAs introduced by coercion"
Warning message in model.matrix.default(mt, mf, contrasts):
"the response appeared on the right-hand side and was dropped"
Warning message in model.matrix.default(mt, mf, contrasts):
"problem with term 6 in model.matrix: no columns are assigned"
Warning message in predict.lm(reg, df_reg_imp[is.na(V7)]):
"prediction from a rank-deficient fit may be misleading"


## Imputation with Perturbed Linear Regression

In [90]:
df_reg_imp_per <- copy(df)
df_reg_imp_per[, V7:=as.numeric(V7)]

# V1 is ID and V11 is response
reg <- lm(V7 ~ V2 + V3 + V4 + V5 + V6 + V7 + V8 + V9 + V10, data=df_reg_imp, na.action=na.omit)
e_sd <- reg$residuals %>% sd
e_N <- df_reg_imp_per[is.na(V7), .N]
e_pur <- rnorm(e_N, mean=0, sd=e_sd)
df_reg_imp_per[is.na(V7), V7:=predict(reg, df_reg_imp_per[is.na(V7)]) + e_pur]

Warning message in eval(jsub, SDenv, parent.frame()):
"NAs introduced by coercion"
Warning message in model.matrix.default(mt, mf, contrasts):
"the response appeared on the right-hand side and was dropped"
Warning message in model.matrix.default(mt, mf, contrasts):
"problem with term 6 in model.matrix: no columns are assigned"
Warning message in predict.lm(reg, df_reg_imp_per[is.na(V7)]):
"prediction from a rank-deficient fit may be misleading"


# Question 15.1

Describe a situation or problem from your job, everyday life, current events, etc., for which optimization would be appropriate. What data would you need? 

# Answers to 15.1

Optimization would be appropriate for robotic agents such as self-driving cars, video game AI and vacuum cleaner robots. I think in both cases the training for the AI models largely involves the reinforcement learning framework. In essence, we want to find optimal actions for each state according to the following dynamic programming model:

\begin{align*}
V(x) = \max_a \Big\{ r(x,a) + \mathbb{E}_{x'}\big[V(x') | x,a\big] \Big\}
\end{align*}

Intuitively, $x$ is the current environment state, $a$ represents an action, $x'$ is a random variable representing consequent states. The functions $r$ and $V$ are the instant reward that the learner gets and the value function that determines the how valuable each environment state is. For example, in Super Mario, the player on top of a pipe $x$ can either move on $a$, resulting in next frame in the forward direction, some [koopa](https://www.mariowiki.com/Koopa_(species)) to beat, and same number of coins ($x'$) or go inside $a$, resulting in another map, no monsters, and more coins (a more favorable $x'$, with higher value $V(x')$).

The data included in state $x$ is required to be able to adequately dsecribe the environment. In the example of self-driving cars, $x$ can include lane position, drivable zone, location of agents such as cars & pedestrians, state of traffic lights and others (as enumerated by [this paper](https://arxiv.org/pdf/2002.00444.pdf)). For game AI, the state variables could include user inputs, previous frames, and predicted frames.

Ultimately, we want to learn $V$ and the optimal action $a^*(x)$ at each given $x$. The dimension and support of $x$ are typically very large rendering the need for reinforcement learning (rather than resorting to closed form solutions). $r$ is carefully pre-defined to appropriately reward the learner for optimal actions. When training the learner, we deploy the learner in the environment and each experience contribute to updates for $V$ and $a^*$. We'd want the learner to pass through as much situations $x$ as possible for completeness ("exploration"), while doing optimal actions to learn the optimal paths ("exploitation"). For example, one may assign a random draw of 5% explore environment randomly, and 95% using optimal action from accumulated experience and decrease the probability gradually. Collecting enough data in $x$ to describe the environment, appropriately defining instant rewards $r$, and balancing exploration vs exploitation are typically nontrivial and requires field knowledge and experience.